In [16]:
import os
import re
import numpy as np

from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

In [31]:
# load data
text = open(os.path.join(os.getcwd(), 'data/shakespeare.txt')).read()

# remove sonnet numbers and convert to lowercase
text = re.sub(r'[0-9]+', '', text) 
text = text.lower()

# create mapping of unique chars to integers
chars = sorted(list(set(text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

n_chars = len(text) # total length of dataset
n_vocab = len(chars) # number of unique characters

In [34]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 40
dataX = []
dataY = []

# use sliding window approach
for i in range(0, n_chars - seq_length, 1):
    seq_in = text[i:i + seq_length]
    seq_out = text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
    
n_sequences = len(dataX)

In [38]:
# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (n_sequences, seq_length, 1))

# normalize data to range (0, 1)
X = X / float(n_vocab)

# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [39]:
# define the LSTM model 
# (single layer with 150 units, followed by dense output layer)
model = Sequential()
model.add(LSTM(150, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [40]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [ ]:
# train the model
model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)

Epoch 1/20
97635/97635 [==============================] - 92s 941us/step - loss: 1.9434

Epoch 00001: loss improved from 1.95374 to 1.94343, saving model to weights-improvement-01-1.9434.hdf5
Epoch 2/20
97635/97635 [==============================] - 86s 878us/step - loss: 1.9375

Epoch 00002: loss improved from 1.94343 to 1.93755, saving model to weights-improvement-02-1.9375.hdf5
Epoch 3/20
97635/97635 [==============================] - 85s 867us/step - loss: 1.9350

Epoch 00003: loss improved from 1.93755 to 1.93497, saving model to weights-improvement-03-1.9350.hdf5
Epoch 4/20
97635/97635 [==============================] - 111s 1ms/step - loss: 1.9260

Epoch 00004: loss improved from 1.93497 to 1.92599, saving model to weights-improvement-04-1.9260.hdf5
Epoch 5/20
97635/97635 [==============================] - 105s 1ms/step - loss: 1.9197

Epoch 00005: loss improved from 1.92599 to 1.91966, saving model to weights-improvement-05-1.9197.hdf5
Epoch 6/20
97635/97635 [==================